In [ ]:
import numpy as np 
import os
import conlleval

train_dir = "dataset/train"
test_dir = "dataset/dev.in"

START_STATE_KEY = "START"
STOP_STATE_KEY = "STOP"

LARGE_NEG = -2**52

In [ ]:
def tokenize(file_path):  
    data, lst = [], []
    with open(file_path, 'r') as f:  
        for line in f:
            if line== '\n':
                data.append(lst)
                lst = []    
            else:
                lines = line.replace("\n",'').split(" ")
                lst.append(tuple(lines))
    return data

train_sentences = tokenize(train_dir)
print(train_sentences[:5])      


# PART 1
## 1i)

In [ ]:

def MLE_emission_parameters(train_sentences):
    ''' Calculates the emission parameters by count(y->x)/count(y)
    
    :param train_sentences: our train file tokenised sentences
    :type train_sentences: list(tuple())

    :return count_y_dict: Count of labels 
    :rtype: dict()

    :return count_y_to_x_dict: Count of words and labels
    :rtype: dict()

    :param emission_dict: value of Count(labels->words)/Count(labels), keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :rtype: dict

    '''

    count_y_dict = {}
    count_y_to_x_dict = {}
    emission_dict = {}
    word_bank = []

    for sentence in train_sentences:
        for x_y_pair in sentence:
            word, label = x_y_pair
            word_bank.append(word)
            count_y_dict[label] = count_y_dict.get(label,0) + 1
            count_y_to_x_dict[(label,word)] = count_y_to_x_dict.get((label,word),0) + 1

    # Calculate our emission
    for key, value in count_y_to_x_dict.items(): 
        label = key[0]
        word = key[1]
        string = f"emission: {label}+{word}" 
        prob =  value / count_y_dict.get(label)
        emission_dict[string] = float(np.where(prob != 0, np.log(prob), 0))

    # handle missing possible words and labels
    unique_word_bank, labels = set(word_bank), count_y_dict.keys()
    for label in labels:       
        for word in unique_word_bank:
            string = f"emission: {label}+{word}"
            if string not in emission_dict:
                emission_dict[string] = LARGE_NEG

    return count_y_dict, count_y_to_x_dict, emission_dict

In [ ]:
count_y_dict, count_y_to_x_dict, emission_dict = MLE_emission_parameters(train_sentences)
print(list(emission_dict.items())[0:5])

## 1ii)

In [ ]:
def  MLE_transition_parameters(train_dir, emission_dict):
    ''' Calculates the transition parameters by count(y->y-1)/count(y)

    :param train_dir: our train file
    :type train_sentences: str

    :param emission_dict: Count(y->x)/Count(y), keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :type emission_dict: dict()

    :return count_y_to_y_dict: Count of labels and previous label
    :rtype: dict()

    :return emission_transition_dict: value of Count(labels->words)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :rtype: dict()
    '''
    count_y_dict = {}
    count_y_to_y_dict = {}
    prev_label = ""
    emission_transition_dict = emission_dict

    with open(train_dir, "r", encoding="utf8") as f:
        for line in f:
            # Parse each line
            if len(line.split(" ")) == 2:
                _, label = line.replace("\n","").split(" ")
            else:
                label = ''
            if label == '' and prev_label != '':
                count_y_dict[STOP_STATE_KEY] = count_y_dict.get(STOP_STATE_KEY) + 1 if count_y_dict.get(STOP_STATE_KEY) else 1
            elif label !='':
                if prev_label == '':
                    count_y_dict[START_STATE_KEY] = count_y_dict.get(START_STATE_KEY) + 1 if count_y_dict.get(START_STATE_KEY) else 1
                if label in count_y_dict:
                    count_y_dict[label] = count_y_dict.get(label)+1
                else:
                    count_y_dict[label] = 1
            if prev_label == '' and label != '':
                if (START_STATE_KEY, label) in count_y_to_y_dict:
                    count_y_to_y_dict[(START_STATE_KEY, label)] = count_y_to_y_dict.get((START_STATE_KEY, label)) + 1
                else:
                    count_y_to_y_dict[(START_STATE_KEY, label)] = 1
            elif label == '' and prev_label != '':
                if (prev_label, STOP_STATE_KEY) in count_y_to_y_dict:
                    count_y_to_y_dict[(prev_label, STOP_STATE_KEY)] = count_y_to_y_dict.get((prev_label, STOP_STATE_KEY)) + 1
                else:
                    count_y_to_y_dict[(prev_label, STOP_STATE_KEY)] = 1
            elif label != '' and prev_label != '':
                if (prev_label, label) in count_y_to_y_dict:
                    count_y_to_y_dict[(prev_label, label)] = count_y_to_y_dict.get((prev_label, label)) + 1
                else:
                    count_y_to_y_dict[(prev_label, label)] = 1
            prev_label = label

    # Calculate our transition
    for key, value in count_y_to_y_dict.items(): # Default is iterate keys()
        prev_label = key[0]
        label = key[1]
        string = f"transition: {prev_label}+{label}" 
        prob =  value / count_y_dict.get(prev_label)
        emission_transition_dict[string] = float(np.where(prob != 0, np.log(prob), 0))
    # print("MLE: \n",list(emission_transition_dict.items()), len(emission_transition_dict) ,"\n")

    labels = count_y_dict.keys()
    for label in labels:
        for prev_label in labels:
            string = f"transition: {prev_label}+{label}" 
            if string not in emission_transition_dict:
                emission_transition_dict[string] = LARGE_NEG

    return count_y_to_y_dict, emission_transition_dict

In [ ]:
count_y_to_y_dict, emission_transition_dict = MLE_transition_parameters(train_dir, emission_dict)
print(list(emission_transition_dict.items())[:5])
print(list(emission_transition_dict.items())[-5:])

# Part 2

## 2i)

In [ ]:
def score(sentence, emission_transition_dict):
    ''' Calculates the score with of a given pair based on emission and transmission features
    
    :param sentences: our  file tokenised sentences
    :type sentences: list(tuple())

    :return emission_transition_dict: value of Count(labels->words)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :type emission_transition_dict: dict()

    :param score: our emission score + transition score for sentence
    :type sentences: float
    '''
    score = 0
    emission_score = 0 
    transition_score = 0
    x_seq = [x[0] for x in sentence]
    y_seq = [START_STATE_KEY]+[y[1] for y in sentence]+[STOP_STATE_KEY]
    
    for i in range(len(x_seq)):
        label = y_seq[i+1]
        word = x_seq[i]
        key = f"emission: {label}+{word}" 
        emission_score += emission_transition_dict[key]
    for j in range(1, len(y_seq)):
        prev_label = y_seq[j-1]
        label = y_seq[j]
        key = f"transition: {prev_label}+{label}" 
        transition_score += emission_transition_dict[key]
    score = emission_score + transition_score
    return score



In [ ]:
score(train_sentences[0],emission_transition_dict)

## 2ii)

In [ ]:
test_sentences = tokenize(test_dir)
print(test_sentences[:5])   

In [ ]:
def viterbi_algo(test_sentences, count_y_dict, emission_transition_dict, output_name):
    ''' Decoding process that finds globally finds the best possible labels from past MLE scores, saves file to output folder
    
    :param test_sentences: our file tokenised sentences
    :type test_sentences: list(tuple())

    :param count_y_dict: Count of labels 
    :param count_y_dict: dict()

    :param emission_transition_dict: value of Count(labels->words)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :param emission_transition_dict: dict()
    '''
    
    pi = [{}]
    path = {}
    labels = count_y_dict.keys()
    os.makedirs('output',exist_ok=True)

    with open(f'output/{output_name}', "w") as outfile:
        for sentence in test_sentences:
            # j = 0 (START)
            for label in labels:
                pi[0][label] = emission_transition_dict.get(f"transition: {START_STATE_KEY}+{label}",LARGE_NEG) + emission_transition_dict.get(f"emission: {label}+{sentence[0][0]}",LARGE_NEG)
                path[label] = [label]
            # j = 1 to N-1
            for idx in range(1,len(sentence)):
                pi.append({})
                newpath = {}
                for label_y in labels:
                    (prob, label) = max([(pi[idx-1][prev_label] + emission_transition_dict.get(f"transition: {prev_label}+{label_y}",LARGE_NEG) + emission_transition_dict.get(f"emission: {label_y}+{sentence[idx][0]}",LARGE_NEG), prev_label) 
                                    for prev_label in labels])
                    pi[idx][label_y] = prob
                    newpath[label_y] = path[label] + [label_y]
                path = newpath
            # j = N (STOP)
            idx = len(sentence)
            (prob, label) = max([(pi[idx-1][label_y] + emission_transition_dict.get(f"transition: {label_y}+{STOP_STATE_KEY}", LARGE_NEG), label_y) for label_y in labels])
            
            # handle inconsistent length
            if len(sentence) != len(path[label]):
                print(len(sentence),len(path[label]))
                raise Exception("{} has a different lenght with {}".format(sentence, path[label]))
            
            # write to file
            for i in range(len(sentence)):
                line = "{} {}\n".format(sentence[i][0], path[label][i])
                outfile.write(line)
                
            outfile.write("\n")

In [ ]:
viterbi_algo(test_sentences, count_y_dict, emission_transition_dict, output_name='dev.p2.out')

# Evaluation of dev.p2.out

In [ ]:
prediction_dir = 'output/dev.p2.out'
truth_dir = 'dataset/dev.out'

def evaluate_results(truth_dir,prediction_dir):
    predictions = []
    prediction_sentences = tokenize(prediction_dir)
    for sentence in prediction_sentences:
        for word_pair in sentence:
            predictions.append(word_pair[1])     
    lines = """"""
    idx = 0
    with open(truth_dir, "r", encoding="utf8") as tstr:
        for line in tstr:
            if len(line) > 1:
                newline = line.replace("\n",f" {predictions[idx]}\n")
                lines += newline
                idx += 1
            else:
                lines += "\n"
    return lines.splitlines()

lines = evaluate_results(truth_dir,prediction_dir)
res = conlleval.evaluate(lines)
print(conlleval.report(res))

# Part 3
## 3i)

In [ ]:
def logSumExp(a):
    max = np.max(a)
    sumOfExp = np.exp(a - max).sum()
    return max + np.log(sumOfExp)

def forward_algorithm(sentence, count_y_dict, emission_transition_dict):    
    pi = [{}]
    labels = count_y_dict.keys()
    # j = 0 (START)
    for label in labels:
        pi[0][label] = emission_transition_dict.get(f"transition: {START_STATE_KEY}+{label}",LARGE_NEG) + emission_transition_dict.get(f"emission: {label}+{sentence[0][0]}",LARGE_NEG)

    # j = 1 to N-1
    for idx in range(1,len(sentence)):
        pi.append({})

        for label in labels:
            log_a = []
            for prev_label in labels:
                log_a.append(pi[idx-1][prev_label] + emission_transition_dict.get(f"transition: {prev_label}+{label}",LARGE_NEG) + emission_transition_dict.get(f"emission: {label}+{sentence[idx][0]}",LARGE_NEG))
            pi[idx][label] = logSumExp(log_a)
            
    # j = N (STOP)
    idx = len(sentence)
    log_a = []
    for label in labels:
        log_a.append(pi[idx-1][label] + emission_transition_dict.get(f"transition: {label}+{STOP_STATE_KEY}", LARGE_NEG))
    return pi, logSumExp(log_a)

forward_algorithm(train_sentences[0], count_y_dict, emission_transition_dict)

In [ ]:
def loss_fn(sentences, count_y_dict, emission_transition_dict):
    loss = 0
    for sent in sentences:
        loss+= score(sent, emission_transition_dict)
        _, update = forward_algorithm(sent, count_y_dict, emission_transition_dict)
        loss-= update
    return (-1)*loss
loss_fn(train_sentences, count_y_dict, emission_transition_dict)

## 3ii)

In [ ]:
def backward_algorithm(sentence, count_y_dict, emission_transition_dict):    
    pi = [{} for i in range(len(sentence))]
    labels = count_y_dict.keys()

    # j = N (STOP)
    for label in labels:
        pi[len(sentence)-1][label] = emission_transition_dict.get(f"transition: {label}+{STOP_STATE_KEY}", LARGE_NEG)

    # j = N-1 to 1 
    for idx in range(len(sentence)-1,0,-1):
        for label in labels:
            log_b = []
            for next_label in labels:
                log_b.append(pi[idx][next_label] + emission_transition_dict.get(f"transition: {label}+{next_label}",LARGE_NEG) + emission_transition_dict.get(f"emission: {next_label}+{sentence[idx][0]}",LARGE_NEG))
            pi[idx-1][label] = logSumExp(log_b)

    # j = 0 (START)
    log_b = []
    for label in labels:
        log_b.append(pi[0][label] + emission_transition_dict.get(f"transition: {START_STATE_KEY}+{label}",LARGE_NEG) + emission_transition_dict.get(f"emission: {label}+{sentence[0][0]}",LARGE_NEG))
    
    return pi, logSumExp(log_b)

In [ ]:
backward_algorithm(train_sentences[0], count_y_dict, emission_transition_dict)

In [ ]:
def forward_backward_algorithm(sentence, count_y_dict, emission_transition_dict):
    feature_expectation = {}
    labels = count_y_dict.keys()
    fwd_pi, fwd_score = forward_algorithm(sentence, count_y_dict, emission_transition_dict)
    bkd_pi, bkd_score = backward_algorithm(sentence, count_y_dict, emission_transition_dict)
        
    # idx = 1
    for label in labels:
        string_transition = f"transition: {START_STATE_KEY}+{label}"
        string_emission = f"emission: {label}+{sentence[0][0]}" 

        # update transition features
        update = bkd_pi[0][label] \
                + emission_transition_dict[string_transition] \
                + emission_transition_dict[string_emission] \
                - fwd_score
         
        feature_expectation[string_transition] = feature_expectation.get(string_transition,0) + np.exp(update)
        
        # update emission features
        update = fwd_pi[0][label] + bkd_pi[0][label] - fwd_score
        feature_expectation[string_emission] = feature_expectation.get(string_emission,0) + np.exp(update)
        
    # idx = 2 to N-1
    for idx in range(1,len(sentence)):
        for label in labels:
            string_emission = f"emission: {label}+{sentence[idx][0]}" 

            # update transition features
            for prev_label in labels:
                string_transition = f"transition: {prev_label}+{label}" 
                update = fwd_pi[idx-1][prev_label] \
                        + bkd_pi[idx][label] \
                        + emission_transition_dict[string_transition] \
                        + emission_transition_dict[string_emission] \
                        - fwd_score
                feature_expectation[string_transition] = feature_expectation.get(string_transition,0) + np.exp(update)

            # update emission features
            update = fwd_pi[idx][label] + bkd_pi[idx][label] - fwd_score      
            feature_expectation[string_emission] = feature_expectation.get(string_emission,0) + np.exp(update)
                
    # idx = N (STOP)
    idx = len(sentence)
    for label in labels:
        # update transition features
        string_transition = f"transition: {label}+{STOP_STATE_KEY}" 
        update = fwd_pi[idx-1][label] + emission_transition_dict[string_transition] - fwd_score
        feature_expectation[string_transition] = feature_expectation.get(string_transition,0) + np.exp(update)
        
    return feature_expectation
                
forward_backward_algorithm(train_sentences[0], count_y_dict, emission_transition_dict)

In [ ]:
def get_features(sentences, count_y_dict, emission_transition_dict):
    features = {k:0 for k,v in emission_transition_dict.items()}
    for sent in sentences:
        expect = forward_backward_algorithm(sent, count_y_dict, emission_transition_dict)
        for k,v in expect.items():
            features[k] += v
    return features

features = get_features(train_sentences, count_y_dict, emission_transition_dict)

In [ ]:
def mapping_fn(emission_transition_dict):
    index_map = {}
    for idx, value in enumerate(emission_transition_dict): 
        # print(value.split(" ")[1].split("+")[0], value.split(" ")[1].split("+")[1])
        if len(value.split(" ")[1].split("+")) > 2:
            first = str(value.split(" ")[1].split("+")[0])
            second = '+'
            print(idx)
        else:
            first = str(value.split(" ")[1].split("+")[0])
            second = str(value.split(" ")[1].split("+")[1])
        index_map[idx] = (first,second)
    return index_map

index_map = mapping_fn(emission_transition_dict)

In [ ]:
def compute_grad(sentences, count_y_dict, features, index_map, emission_transition_dict):
    labels = count_y_dict.keys()
    counter = 0 
    grad_lst = np.zeros(len(emission_transition_dict),)
    for i in range(len(features)):
        # print(index_map[i],count_y_to_x_dict.keys())
        if index_map[i] in count_y_to_x_dict.keys():
            string = f'emission: {index_map[i][0]}+{index_map[i][1]}'
            grad_lst[i] += (features[string] - count_y_to_x_dict[index_map[i]])
        elif index_map[i] in count_y_to_y_dict.keys():
            string = f'transition: {index_map[i][0]}+{index_map[i][1]}'
            grad_lst[i] += (features[string] - count_y_to_y_dict[index_map[i]])
        else:
            try:
                string = f'emission: {index_map[i][0]}+{index_map[i][1]}'
                grad_lst[i] += (features[string])
            except:
                pass
            try:
                string = f'transition: {index_map[i][0]}+{index_map[i][1]}'
                grad_lst[i] += (features[string])
            except:
                pass
    return grad_lst

compute_grad(train_sentences, count_y_dict, features, index_map, emission_transition_dict)

# PART 4
## 4i)

In [ ]:
def loss_with_reg(w, sentences, count_y_dict, emission_transition_dict, n = 0.1):
    loss = loss_fn(sentences, count_y_dict, emission_transition_dict)
    # regularization
    loss += n*sum(w1*w1 for w1 in w)
    return loss

In [ ]:
def grad_with_reg(w, sentences, count_y_dict, features, index_map, emission_transition_dict, n = 0.1):
    grad_lst = compute_grad(sentences, count_y_dict, features, index_map, emission_transition_dict)
    for i in range(len(w)):
        grad_lst[i] += w[i]*2*n
    return grad_lst

In [ ]:
import time
from scipy.optimize import fmin_l_bfgs_b
total_start = time.time()

def callbackF(w):
    '''
    This function will only be called by "fmin_l_bfgs_b"
    Arg:
    w: weights, numpy array
    '''
    loss = get_loss_grad(w)[0]
    print('Loss:{0:.4f}'.format(loss))

def get_loss_grad(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
    w: weights, numpy array
    Returns:
    loss: loss, float
    grads: gradients, numpy array
    '''
    # to be completed by you
    start = time.time()
    new_emission_transition_dict = {}
    for i in range(len(index_map.keys())):
        if i < len(emission_dict):
            string = f'emission: {index_map[i][0]}+{index_map[i][1]}'
            new_emission_transition_dict[string] = w[i]
        else:
            string = f'transition: {index_map[i][0]}+{index_map[i][1]}'
            new_emission_transition_dict[string] = w[i]
    features = get_features(train_sentences, count_y_dict, new_emission_transition_dict)

    loss = loss_with_reg(w, train_sentences, count_y_dict, new_emission_transition_dict, n = 0.1)
    print('loss: '+ str(loss))
    
    grad_lst = grad_with_reg(w, train_sentences, count_y_dict, features, index_map, new_emission_transition_dict, n = 0.1)
    #print('grad_lst: '+ str(grad_lst))
    grads = np.asarray(list(grad_lst)) 
    
    print('time taken: '+ str(time.time()-start) +' total time: '+ str(time.time()-total_start))
    
    return loss, grads
  
init_w = np.zeros(len(index_map),)
results = fmin_l_bfgs_b(get_loss_grad, init_w, pgtol=0.01, maxiter=50, callback=callbackF)

## 4ii)

In [ ]:
new_emission_transition_dict = {}
for i in range(len(index_map.keys())): #27899
    if i< len(emission_dict): #27818
        string = f'emission: {index_map[i][0]}+{index_map[i][1]}'
        new_emission_transition_dict[string] = results[0][i]
    else:
        string = f'transition: {index_map[i][0]}+{index_map[i][1]}'
        new_emission_transition_dict[string] = results[0][i]

viterbi_algo(test_sentences, count_y_dict, new_emission_transition_dict, 'dev.p4.out')

In [ ]:
prediction_dir = 'output/dev.p4.out'
truth_dir = 'dataset/dev.out'

lines = evaluate_results(truth_dir,prediction_dir)
res = conlleval.evaluate(lines)
print(conlleval.report(res))

# Part 5

In [ ]:
def unigram_1_parameters(train_dir, emission_dict):
    """Calculates the transition parameters by count(y->x_i-1)/count(y)

    :param train_dir: our train file
    :type train_sentences: str

    :param emission_dict: count(y->x_i-1)/count(y), keys are tuples of word and label ('unigram_1: O+All', -9.01768561), value MLE
    :type emission_dict: dict()

    :return count_y_to_y_dict: Count of labels and previous label
    :rtype: dict()

    :return emission_transition_dict: value of Count(labels->words_i-1)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('unigram_1: O+All', -9.01768561), value MLE
    :rtype: dict()
    """
    # key is label | value is count
    count_y_dict = {}
    # key is word_i-1 , label_i | value is count
    count_y_to_x_dict = {}

    with open(train_dir, "r", encoding="utf8") as f:
        prev_word, prev_label = "", ""
        for line in f:
            # Parse each line
            if len(line.split(" ")) == 2:
                word, label = line.replace("\n", "").split(" ")
            else:
                label = ""

            # counting
            if label == "" and prev_label != "":
                count_y_dict[STOP_STATE_KEY] = count_y_dict.get(STOP_STATE_KEY, 0) + 1

            elif label != "":
                if prev_label == "":
                    count_y_dict[START_STATE_KEY] = (
                        count_y_dict.get(START_STATE_KEY, 0) + 1
                    )
                if label in count_y_dict:
                    count_y_dict[label] = count_y_dict.get(label) + 1
                else:
                    count_y_dict[label] = 1

            # Counting unigram
            if label != "" and prev_word != "":
                count_y_to_x_dict[(label, prev_word)] = (
                    count_y_to_x_dict.get((label, prev_word), 0) + 1
                )

            prev_word, prev_label = word, label

    # Calculate unigram
    for key, value in count_y_to_x_dict.items():  # Default is iterate keys()
        label = key[0]
        word = key[1]
        string = f"unigram_1: {label}+{word}"

        prob = value / count_y_dict.get(label)
        emission_dict[string] = float(np.where(prob != 0, np.log(prob), LARGE_NEG))

    print(
        "unigram_1 yi -> xi-1: \n",
        list(emission_dict.items())[-10:],
        len(emission_dict),
        "\n",
    )
    emission_transition_dict = emission_dict

    return count_y_to_x_dict, emission_transition_dict


In [ ]:
def unigram_2_parameters(train_dir, emission_dict):
    """Calculates the transition parameters by count(y->x_i+1)/count(y)

    :param train_dir: our train file
    :type train_sentences: str

    :param emission_dict: count(y->x_i+1)/count(y), keys are tuples of word and label ('unigram_1: O+All', -9.01768561), value MLE
    :type emission_dict: dict()

    :return count_y_to_y_dict: Count of labels and previous label
    :rtype: dict()

    :return emission_transition_dict: value of Count(labels -> words_i+1)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('unigram_1: O+All', -9.01768561), value MLE
    :rtype: dict()
    """
    # key is label | value is count
    count_y_dict = {}
    # key is word_i+1 , label_i | value is count
    count_y_to_x_dict = {}

    with open(train_dir, "r", encoding="utf8") as f:
        prev_word, prev_label = "", ""
        for line in f:
            # Parse each line
            if len(line.split(" ")) == 2:
                word, label = line.replace("\n", "").split(" ")
            else:
                label = ""

            # counting
            if label == "" and prev_label != "":
                count_y_dict[STOP_STATE_KEY] = count_y_dict.get(STOP_STATE_KEY, 0) + 1
            elif label != "":
                if prev_label == "":
                    count_y_dict[START_STATE_KEY] = (
                        count_y_dict.get(START_STATE_KEY, 0) + 1
                    )
                if label in count_y_dict:
                    count_y_dict[label] = count_y_dict.get(label) + 1
                else:
                    count_y_dict[label] = 1

            if prev_label != "" and word != "":
                count_y_to_x_dict[(prev_label, word)] = (
                    count_y_to_x_dict.get((prev_label, word), 0) + 1
                )

            prev_word, prev_label = word, label

    # Calculate unigram
    for (label, word), value in count_y_to_x_dict.items():  # Default is iterate keys()
        if prev_label != "" and word != "":
            string = f"unigram_2: {label}+{word}"
            prob = value / count_y_dict.get(label)
            emission_dict[string] = float(np.where(prob != 0, np.log(prob), LARGE_NEG))

    print(
        "unigram_2 yi -> x_i+1: \n",
        list(emission_dict.items())[-10:],
        len(emission_dict),
        "\n",
    )
    emission_transition_dict = emission_dict

    return count_y_to_x_dict, emission_transition_dict


In [ ]:
def bigram_parameters(train_dir, emission_dict):
    """Calculates the transition parameters by count(y->x_i+1)/count(y)

    :param train_dir: our train file
    :type train_sentences: str

    :param emission_dict: count(yi-1 -> yi -> xi)/count(y), keys are tuples of word and label ('B-neutral+O+B-neutral', -9.01768561)
    :type emission_dict: dict()

    :return count_y_to_y_dict: Count of labels and previous label
    :rtype: dict()

    :return emission_transition_dict: value of Count(label-1 -> labels -> words)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('B-neutral+O+B-neutral', -9.01768561)
    :rtype: dict()
    """
    # key is label | value is count
    count_y_dict = {}
    # key is word_i+1 , label_i | value is count
    count_y_to_y_to_x_dict = {}

    with open(train_dir, "r", encoding="utf8") as f:
        prev_word, prev_label = "", ""
        for line in f:
            # Parse each line
            if len(line.split(" ")) == 2:
                word, label = line.replace("\n", "").split(" ")
            else:
                label = ""

            # counting
            if label == "" and prev_label != "":
                count_y_dict[STOP_STATE_KEY] = count_y_dict.get(STOP_STATE_KEY, 0) + 1
            elif label != "":
                if prev_label == "":
                    count_y_dict[START_STATE_KEY] = (
                        count_y_dict.get(START_STATE_KEY, 0) + 1
                    )
                if label in count_y_dict:
                    count_y_dict[label] = count_y_dict.get(label) + 1
                else:
                    count_y_dict[label] = 1

            if prev_label != "" and word != "" and label != "":
                count_y_to_y_to_x_dict[(prev_label, label, word)] = (
                    count_y_to_y_to_x_dict.get((prev_label, label, word), 0) + 1
                )

            prev_label = label

    # Calculate unigram
    for key, value in count_y_to_y_to_x_dict.items():  # Default is iterate keys()
        prev_label, label, word = key
        if prev_label != "" and label != "" and word != "":
            string = f"bigram: {prev_label}+{label}+{word}"
            prob = value / count_y_dict.get(label)
            emission_dict[string] = float(np.where(prob != 0, np.log(prob), LARGE_NEG))
        prev_label = label

    print(
        "bigram yi-1 -> yi -> xi: \n",
        list(emission_dict.items())[-10:],
        len(emission_dict),
        "\n",
    )
    emission_transition_dict = emission_dict

    return count_y_to_y_to_x_dict, emission_transition_dict


In [ ]:
_, _, emission_dict = MLE_emission_parameters(train_sentences)
_, emission_dict = MLE_transition_parameters(train_dir, emission_dict)
_, emission_dict = unigram_1_parameters(train_dir, emission_dict)
_, emission_dict = unigram_2_parameters(train_dir, emission_dict)
count_y_dict, emission_dict = bigram_parameters(train_dir, emission_dict)

print(list(emission_dict.items())[:5])
print(list(emission_dict.items())[-5:])


In [ ]:
def viterbi_algo_2(test_sentences, count_y_dict, emission_dict):
    """Decoding process that finds greedily finds the best possible labels from past MLE scores, saves file to output folder

    :param test_sentences: our file tokenised sentences
    :type test_sentences: list(tuple())

    :param count_y_dict: Count of labels
    :param count_y_dict: dict()

    :param emission_dict: value of Count(labels->words)/Count(labels) for emission and Count(prev_labels->labels)/Count(labels) for transmission, keys are tuples of word and label ('emission: O+All', -9.01768561), value MLE
    :param emission_dict: dict()
    """

    pi = [{}]
    path = {}
    labels = count_y_dict.keys()
    os.makedirs("output", exist_ok=True)

    with open("output/dev.p5.out", "w") as outfile:
        for sentence in test_sentences:
            # j = 0 (START)
            for label in labels:
                pi[0][label] = emission_dict.get(
                    f"transition: {'START'}+{label}", LARGE_NEG
                ) + emission_dict.get(f"emission: {label}+{sentence[0][0]}", LARGE_NEG)
                path[label] = [label]
            # j = 1 to N-1
            for idx in range(1, len(sentence)):
                pi.append({})
                newpath = {}
                for label_y in labels:
                    (prob, label) = max(
                        [
                            (
                                pi[idx - 1][prev_label]
                                + emission_dict.get(
                                    f"transition: {prev_label}+{label_y}", LARGE_NEG
                                )
                                + emission_dict.get(
                                    f"emission: {label_y}+{sentence[idx][0]}", LARGE_NEG
                                )
                                + (
                                    emission_dict.get(
                                        f"unigram_1: {label_y}+{sentence[idx-1][0]}",
                                        LARGE_NEG,
                                    )
                                )
                                + (
                                    emission_dict.get(
                                        f"unigram_2: {label_y}+{sentence[idx+1][0]}",
                                        LARGE_NEG,
                                    )
                                    if idx < len(sentence) - 1
                                    else 0
                                )
                                + emission_dict.get(
                                    f"bigram: {prev_label}+{label_y}+{sentence[idx][0]}",
                                    LARGE_NEG,
                                ),
                                prev_label,
                            )
                            for prev_label in labels
                        ]
                    )
                    pi[idx][label_y] = prob
                    newpath[label_y] = path[label] + [label_y]
                path = newpath
            # j = N (STOP)
            idx = len(sentence)
            (prob, label) = max(
                [
                    (
                        pi[idx - 1][label_y]
                        + emission_dict.get(
                            f"transition: {label_y}+{'STOP'}", LARGE_NEG
                        ),
                        label_y,
                    )
                    for label_y in labels
                ]
            )

            # handle inconsistent length
            if len(sentence) != len(path[label]):
                print(len(sentence), len(path[label]))
                raise Exception(
                    "{} has a different lenght with {}".format(sentence, path[label])
                )

            # write to file
            for i in range(len(sentence)):
                line = f"{sentence[i][0]} {path[label][i]}\n"
                outfile.write(line)

            outfile.write("\n")


In [ ]:
viterbi_algo_2(test_sentences, count_y_dict, emission_dict)

lines = evaluate_results("dataset/dev.out", "output/dev.p5.out")
res = conlleval.evaluate(lines)
print(conlleval.report(res))
